# ĐỒ ÁN CUỐI KỲ - NHẬP MÔN KHOA HỌC DỮ LIỆU

## Đề tài: Ước lượng giá laptop

### Sinh viên:
        1. MSSV: 18120058        Tên: Phạm Công Minh
        2. MSSV: 18120090        Tên: Phạm Nguyên Minh Thy

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
#from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

In [2]:
import sklearn
sklearn.__version__

'0.23.1'

In [3]:
data_df = pd.read_csv('data.csv')
data_df.head()

,producer,processor prod,processor model,cores,core base speed (GHz),ram type,ram cap (GB),ssd (GB),hdd (GB),gpu prod,gpu size (MB),screen type,screen size (inch),weight (kg),os,price(USD)
0,apple,intel,i7,6,2.6,ddr4,16,0,256,amd,4096,led ips,15.4,1.83,macos 11.00,2499.5
1,apple,intel,i3,2,1.1,ddr4,8,0,256,intel,8192,led ips,13.3,1.25,macos 11.00,799.5
2,apple,intel,i5,4,1.4,ddr3,8,0,256,intel,128,led ips,13.3,1.37,macos 11.00,1049.0
3,apple,intel,i7,6,2.6,ddr4,16,0,512,amd,4096,led ips,16.1,2.00,macos 11.00,2199.5
4,apple,intel,i5,4,2.0,ddr4,16,0,512,intel,8192,led ips,13.3,1.40,macos 11.00,1699.5


## Ý nghĩa các cột

    1. producer                 tên nhà sản xuất laptop
    2. processor prod           nhà sản xuất vi xử lý
    3. processor model          mẫu vi xử lý
    4. cores                    số nhân của vi xử lý
    5. core base speed (GHz)    tốc độ cơ bản của vi xử lý
    6. ram type                 loại RAM
    7. ram cap (GB)             dung lượng RAM
    8. ssd (GB)                 dung lượng SSD
    9. hdd (GB)                 dung lượng HDD
    10. gpu prod                nhà sản xuất card màn hình
    11. gpu size (MB)           dung lượng card màn hình
    12. screen type             loại màn hình
    13. screen size (inch)      kích thước màn hình
    14. weight (kg)             trọng lượng máy
    15. os                      hệ điều hành
    16. price(USD)              giá

---

## Khám phá dữ liệu

In [4]:
# Dữ liệu có bao nhiêu dòng, bao nhiêu cột?
data_df.shape

(938, 16)

In [5]:
# Dữ liệu có dòng bị lặp không?
data_df.index.duplicated().sum()

0

In [6]:
# Cột output hiện có kiểu dữ liệu gì?
data_df["price(USD)"].dtype

dtype('float64')

In [7]:
# Cột output có giá trị thiếu không?
data_df["price(USD)"].isna().sum()

0

## Câu hỏi cần trả lời:

**Câu hỏi có dạng:** *Tính giá tiền laptop (`price(USD)`) dựa trên các thuộc tính của laptop*

**Lợi ích khi trả lời câu hỏi:** Dự đoán giá laptop dựa trên các thuộc tính giúp cho việc lựa chọn mẫu khi có dự định mua laptop, có 2 hướng đưa ra quyết định:
- Chọn trước các mẫu phù hợp túi tiền rồi lấy cấu hình tính ra giá dự đoán và so sánh với giá niêm yết của cửa hàng => chọn mẫu laptop có tỷ lệ giá thực tế/giá dự đoán thấp nhất.
- Đưa ra những cấu hình mong muốn để dự đoán giá của máy tính rồi từ khoảng giá tiền đó tìm những mẫu máy tính có những thuộc tính gần với mong muốn.

** Đây chỉ là một trong nhiều yếu tố để quyết định việc chọn mẫu laptop phù hợp.*

**Nguồn cảm hứng của câu hỏi:**
- Việc chọn cách định giá một sản phẩm nào đó lấy cảm hứng từ các bài làm khác đã thấy qua (qua thầy giới thiệu việc tính giá nhà, các bài tập Machine Learning trên mạng,...).
- Việc chọn laptop để dự đoán giá lấy cảm hứng từ chính bản thân khi đã từng phân vân trong việc chọn mẫu laptop để mua và chưa có những cơ sở đủ mạnh để đưa ra quyết định.

---

## Tiền xử lý (tách các tập)

In [8]:
# Tách X và y
y_sr = data_df["price(USD)"]
X_df = data_df.drop("price(USD)", axis=1)

In [9]:
# Tách từ dữ liệu ra tập test theo tỉ lệ 70%:30%
X_df, test_X_df, y_sr, test_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)
# Từ 70% dữ liệu còn lại tách tập train và tập validation theo tỉ lệ 70%:30%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

---

## Khám phá dữ liệu (tập huấn luyện)

In [10]:
# Xem kiểu dữ liệu của các cột
train_X_df.dtypes

producer                  object
processor prod            object
processor model           object
cores                      int64
core base speed (GHz)    float64
ram type                  object
ram cap (GB)               int64
ssd (GB)                   int64
hdd (GB)                   int64
gpu prod                  object
gpu size (MB)              int64
screen type               object
screen size (inch)       float64
weight (kg)              float64
os                        object
dtype: object

Các cột đều có kiểu dữ liệu phù hợp.

In [11]:
# Xét sự phân bố giá trị của các thuộc tính dạng số
num_cols = ['cores', 'core base speed (GHz)', 
            'ram cap (GB)',
            'ssd (GB)', 'hdd (GB)', 
            'gpu size (MB)', 
            'screen size (inch)',
            'weight (kg)']

df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,cores,core base speed (GHz),ram cap (GB),ssd (GB),hdd (GB),gpu size (MB),screen size (inch),weight (kg)
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
min,2.0,1.0,4.0,0.0,0.0,128.0,11.6,0.85
lower_quartile,4.0,1.6,8.0,0.0,256.0,1700.0,14.0,1.40
median,4.0,2.1,12.0,0.0,512.0,3072.0,15.6,1.80
upper_quartile,6.0,2.6,16.0,0.0,512.0,6144.0,15.6,2.20
max,8.0,3.0,128.0,2000.0,8192.0,16384.0,17.3,4.70


In [12]:
# Xét sự phân bố giá trị của các thuộc tính không phải dạng số
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn
cat_cols = list(set(train_X_df.columns) - set(num_cols))
df = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])

,gpu prod,producer,screen type,os,processor prod,processor model,ram type
missing_ratio,0,0,0,0,0,0,0
num_values,3,7,3,6,2,6,2
value_ratios,"{'intel': 56.6, 'nvidia': 33.3, 'amd': 10.0}","{'hp': 23.5, 'lenovo': 23.3, 'asus': 20.7, 'dell': 19.4, 'acer': 8.1, 'lg': 2.6, 'apple': 2.4}","{'led ips': 88.7, 'led tn': 10.0, 'oled': 1.3}","{'windows home 10.00': 57.5, 'windows pro 10.00': 35.1, 'macos 11.00': 2.4, 'chrome os 0.00': 2.2, 'linux ubuntu 0.00': 1.5, 'no os 0.00': 1.3}","{'intel': 86.9, 'amd': 13.1}","{'i5': 38.3, 'i7': 38.1, 'i3': 10.5, 'ryzen 5': 7.8, 'ryzen 7': 3.9, 'ryzen 3': 1.3}","{'ddr4': 96.1, 'ddr3': 3.9}"


In [13]:
# Xét sự phân bố giá trị của Output
train_y_sr.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

missing_ratio        0.0
min                319.0
lower_quartile     779.5
median            1137.0
upper_quartile    1599.8
max               4930.0
Name: price(USD), dtype: float64

Kết luận: Có vẻ là không có gì bất thường.

---

## Tiền xử lý (tập huấn luyện)

Các bước tiền xử lý:
- Tách `processor model` ra thành 2 cột: 1 cột gồm các processor của intel, 1 cột gồm các processor của amd; do cách tách này nên cột `processor prod` không còn cần thiết nữa.
- 2 cột `producer` và `os` có nhiều giá trị khác nhau nên sẽ chọn các giá trị xuất hiện nhiều nhất theo `num_top_producer` và `num_top_os` các giá trị khác được thay bằng giá trị "others".

In [14]:
train_X_df.head()

,producer,processor prod,processor model,cores,core base speed (GHz),ram type,ram cap (GB),ssd (GB),hdd (GB),gpu prod,gpu size (MB),screen type,screen size (inch),weight (kg),os
607,acer,intel,i5,4,1.0,ddr4,8,0,512,intel,8192,led ips,14.0,1.19,windows home 10.00
369,lenovo,intel,i5,4,1.6,ddr4,8,0,256,nvidia,2048,led ips,15.6,1.75,windows pro 10.00
44,asus,intel,i7,6,2.6,ddr4,16,0,1024,nvidia,6144,oled,15.6,2.50,windows pro 10.00
66,asus,intel,i7,4,1.8,ddr4,16,1000,128,intel,1700,led ips,15.6,1.90,windows home 10.00
93,asus,amd,ryzen 7,8,2.9,ddr4,8,0,512,nvidia,4096,led ips,14.0,1.60,windows home 10.00


In [15]:
# Hàm định nghĩa transformer tách giá trị của thuộc tính processor_model thành 2 cột: 
# - 1 cột gồm các processor của intel
# - 1 cột gồm các processor của amd
# Và chuyển về dạng số bằng phương pháp ranking
# Đồng thời xóa 2 cột "processor prod" và "processor model"
def split_processor_model(X):
    intel_processor_rank = {'i3': 1,
                            'i5': 2,
                            'i7': 3,
                            'ryzen 3': 0,
                            'ryzen 5': 0,
                            'ryzen 7': 0}
    intel_processor_model = X['processor model'].replace(intel_processor_rank)


    amd_processor_rank = {'i3': 0,
                          'i5': 0,
                          'i7': 0,
                          'ryzen 3': 1,
                          'ryzen 5': 2,
                          'ryzen 7': 3}
    amd_processor_model = X['processor model'].replace(amd_processor_rank)
    
    return X.assign(intel_processor_model=intel_processor_model, amd_processor_model=amd_processor_model).drop(["processor prod", "processor model"], axis=1)

In [16]:
# TEST
split_processor_model(train_X_df).head()

,producer,cores,core base speed (GHz),ram type,ram cap (GB),ssd (GB),hdd (GB),gpu prod,gpu size (MB),screen type,screen size (inch),weight (kg),os,intel_processor_model,amd_processor_model
607,acer,4,1.0,ddr4,8,0,512,intel,8192,led ips,14.0,1.19,windows home 10.00,2,0
369,lenovo,4,1.6,ddr4,8,0,256,nvidia,2048,led ips,15.6,1.75,windows pro 10.00,2,0
44,asus,6,2.6,ddr4,16,0,1024,nvidia,6144,oled,15.6,2.50,windows pro 10.00,3,0
66,asus,4,1.8,ddr4,16,1000,128,intel,1700,led ips,15.6,1.90,windows home 10.00,3,0
93,asus,8,2.9,ddr4,8,0,512,nvidia,4096,led ips,14.0,1.60,windows home 10.00,0,3


In [17]:
# Hàm chọn các giá trị xuất hiện nhiều nhất theo "num_top_producer" đối với cột "producer" và "num_top_os" đối với cột "os"
# và các giá trị khác được thay bằng giá trị "others"
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_producers=1, num_top_os=1):
        self.num_top_producers = num_top_producers
        self.num_top_os = num_top_os
    def fit(self, X_df, y=None):
        producer_col = X_df.producer
        self.producer_counts_ = producer_col.value_counts()
        producers = list(self.producer_counts_.index)
        self.top_producers_ = producers[:max(1, min(self.num_top_producers, len(producers)))]
        
        os_col = X_df.os
        self.os_counts_ = os_col.value_counts()
        os = list(self.os_counts_.index)
        self.top_os_ = os[:max(1, min(self.num_top_os, len(os)))]
        
        return self
    def transform(self, X_df, y=None):
        X = X_df.copy()
        X.loc[:, "producer"].replace(list(set(X.producer.unique())-set(self.top_producers_)), 'others', inplace=True)
        X.loc[:, "os"].replace(list(set(X.os.unique())-set(self.top_os_)), 'others', inplace=True)
        return X

In [18]:
# TEST
col_adderdropper = ColAdderDropper(num_top_producers=4, num_top_os=4)
col_adderdropper.fit(split_processor_model(train_X_df))
print(col_adderdropper.producer_counts_)
print()
print(col_adderdropper.top_producers_)
print()
print(col_adderdropper.os_counts_)
print()
print(col_adderdropper.top_os_)

hp        108
lenovo    107
asus       95
dell       89
acer       37
lg         12
apple      11
Name: producer, dtype: int64

['hp', 'lenovo', 'asus', 'dell']

windows home 10.00    264
windows pro 10.00     161
macos  11.00           11
chrome os  0.00        10
linux ubuntu 0.00       7
no os  0.00             6
Name: os, dtype: int64

['windows home 10.00', 'windows pro 10.00', 'macos  11.00', 'chrome os  0.00']


In [19]:
fewer_cols_train_X_df = col_adderdropper.transform(split_processor_model(train_X_df))

In [20]:
fewer_cols_train_X_df.os.unique()

array(['windows home 10.00', 'windows pro 10.00', 'macos  11.00',
       'chrome os  0.00', 'others'], dtype=object)

In [21]:
fewer_cols_train_X_df.producer.unique()

array(['others', 'lenovo', 'asus', 'hp', 'dell'], dtype=object)

In [22]:
fewer_cols_train_X_df.head()

,producer,cores,core base speed (GHz),ram type,ram cap (GB),ssd (GB),hdd (GB),gpu prod,gpu size (MB),screen type,screen size (inch),weight (kg),os,intel_processor_model,amd_processor_model
607,others,4,1.0,ddr4,8,0,512,intel,8192,led ips,14.0,1.19,windows home 10.00,2,0
369,lenovo,4,1.6,ddr4,8,0,256,nvidia,2048,led ips,15.6,1.75,windows pro 10.00,2,0
44,asus,6,2.6,ddr4,16,0,1024,nvidia,6144,oled,15.6,2.50,windows pro 10.00,3,0
66,asus,4,1.8,ddr4,16,1000,128,intel,1700,led ips,15.6,1.90,windows home 10.00,3,0
93,asus,8,2.9,ddr4,8,0,512,nvidia,4096,led ips,14.0,1.60,windows home 10.00,0,3



Các bước tiền xử lý tiếp theo: (như Bài tập 3)
- Với các cột dạng số (`nume_cols`), ta sẽ điền giá trị thiếu bằng giá trị mean của cột (dùng `SimpleImputer`). Với *tất cả* các cột dạng số trong tập huấn luyện, ta đều cần tính mean, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới. 
- Với các cột không phải dạng số và không có thứ tự (`unorder_cate_cols`):
    - Ta sẽ điền giá trị thiếu bằng giá trị mode (giá trị xuất hiện nhiều nhất) của cột (dùng `SimpleImputer`). Với *tất cả* các cột không có dạng số và không có thứ tự, ta đều cần tính mode, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới.
    - Sau đó, ta sẽ chuyển sang dạng số bằng phương pháp mã hóa one-hot (dùng `OneHotEncoder`).
- Với cột không phải dạng số và có thứ tự (`order_cate_cols`):
    - Ta sẽ điền giá trị thiếu bằng giá trị mode (giá trị xuất hiện nhiều nhất) của cột.
    - Cột này đã được chuyển sang dạng số rồi nên ta không cần chuyển nữa.
- Cuối cùng, khi tất cả các cột đã được điền giá trị thiếu và đã có dạng số, ta sẽ tiến hành chuẩn hóa bằng cách trừ đi mean và chia cho độ lệch chuẩn của cột để giúp cho các thuật toán cực tiểu hóa như Gradient Descent, LBFGS, ... hội tụ nhanh hơn (dùng `StandardScaler`).

In [23]:
nume_cols = ['cores', 'core base speed (GHz)', 'ram cap (GB)', 'ssd (GB)', 'hdd (GB)', 'gpu size (MB)', 'screen size (inch)', 'weight (kg)']
unorder_cate_cols = ['producer', 'ram type', 'gpu prod', 'screen type', 'os']
order_cate_cols = ['intel_processor_model', 'amd_processor_model']

unorder_cate_cols_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore'))
column_transformer = make_column_transformer((SimpleImputer(strategy='mean'), nume_cols),
                                            (unorder_cate_cols_transformer, unorder_cate_cols),
                                            (SimpleImputer(strategy='most_frequent'), order_cate_cols))
preprocess_pipeline = make_pipeline(FunctionTransformer(split_processor_model), 
                                    ColAdderDropper(num_top_producers=4, num_top_os=4),
                                    column_transformer, StandardScaler())

preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)

In [24]:
preprocessed_train_X

array([[-0.32130204, -2.04938144, -0.54461986, ..., -0.73502956,
        -0.01518628, -0.37218703],
       [-0.32130204, -0.92709143, -0.54461986, ...,  1.3604895 ,
        -0.01518628, -0.37218703],
       [ 1.08996721,  0.94339193,  0.0910585 , ...,  1.3604895 ,
         0.98059951, -0.37218703],
       ...,
       [-0.32130204, -0.92709143, -0.54461986, ...,  1.3604895 ,
        -0.01518628, -0.37218703],
       [ 1.08996721,  0.94339193,  1.36241522, ..., -0.73502956,
         0.98059951, -0.37218703],
       [-0.32130204, -2.04938144, -0.22678068, ..., -0.73502956,
        -0.01518628, -0.37218703]])

In [25]:
preprocess_pipeline

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function split_processor_model at 0x0000022FCDF5AD30>)),
                ('coladderdropper',
                 ColAdderDropper(num_top_os=4, num_top_producers=4)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(),
                                                  ['cores',
                                                   'core base speed (GHz)',
                                                   'ram cap (GB)', 'ssd (GB)',
                                                   'hdd (GB)', 'gpu size (MB...
                                                   'weight (kg)']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['producer', 'ram type',
                                                   'gpu prod', 'screen type',
                                                   'os']),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['intel_processor_model',
                                                   'amd_processor_model'])])),
                ('standardscaler', StandardScaler())])

---

## Mô hình hóa

Sử dụng thuật toán Stochastic Gradient Descent cho bài toán Linear Regression
Chạy thử nghiệm các siêu tham số:
- `alpha`
- `num_top_producers`
- `num_top_os`

Chọn các giá trị của các siêu tham số cho độ lỗi nhỏ nhất trên tập validation

In [26]:
from sklearn.linear_model import SGDRegressor

In [27]:
# Tính độ đo r^2 trên tập huấn luyện
def compute_mse(y, preds):
    return ((y - preds) ** 2).mean()
def compute_rr(y, preds, baseline_preds):
    return 1 - compute_mse(y, preds) / compute_mse(y, baseline_preds)
baseline_preds = train_y_sr.mean()

In [28]:
full_pipeline = make_pipeline(FunctionTransformer(split_processor_model), 
                              ColAdderDropper(num_top_producers=4, num_top_os=4),
                              column_transformer, StandardScaler(),
                              SGDRegressor(penalty='l1', random_state=0))

train_errs = []
val_errs = []
alphas = [0.001, 0.01, 0.1, 1, 10]
num_top_producers_s = range(1, 8)
num_top_os_s = range(1, 7)
best_val_err = float('inf'); best_num_top_os = None; best_num_top_producers = None; best_alpha = None
for alpha in alphas:
    for num_top_producers in num_top_producers_s:
        for num_top_os in num_top_os_s:
            # YOUR CODE HERE
            full_pipeline.set_params(coladderdropper__num_top_producers=num_top_producers, coladderdropper__num_top_os=num_top_os, sgdregressor__alpha=alpha)
            full_pipeline.fit(train_X_df, train_y_sr)
            train_errs.append(100 - compute_rr(train_y_sr, full_pipeline.predict(train_X_df), baseline_preds) * 100)
            val_errs.append(100 - compute_rr(val_y_sr, full_pipeline.predict(val_X_df), baseline_preds) * 100)
            if val_errs[-1] < best_val_err:
                best_val_err = val_errs[-1]
                best_alpha = alpha
                best_num_top_producers = num_top_producers
                best_num_top_os = num_top_os

In [29]:
print(best_val_err)
print(best_alpha)
print(best_num_top_producers)
print(best_num_top_os)

31.2696165003118
10
2
5


## Đánh giá mô hình tìm được

Huấn luyện lại `full_pipeline` trên `X_df` và `y_sr` (tập huấn luyện + tập validation) với `best_alpha`
, `best_num_top_producers` và `best_num_top_os` tìm được ở trên để ra được mô hình cụ thể cuối cùng.

In [30]:
full_pipeline.set_params(coladderdropper__num_top_producers=num_top_producers, coladderdropper__num_top_os=num_top_os, sgdregressor__alpha=alpha)
full_pipeline.fit(X_df, y_sr)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function split_processor_model at 0x0000022FCDF5AD30>)),
                ('coladderdropper',
                 ColAdderDropper(num_top_os=6, num_top_producers=7)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(),
                                                  ['cores',
                                                   'core base speed (GHz)',
                                                   'ram cap (GB)', 'ssd (GB)',
                                                   'hdd (GB)', 'gpu size (MB...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['producer', 'ram type',
                                                   'gpu prod', 'screen type',
                                                   'os']),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['intel_processor_model',
                                                   'amd_processor_model'])])),
                ('standardscaler', StandardScaler()),
                ('sgdregressor',
                 SGDRegressor(alpha=10, penalty='l1', random_state=0))])

In [31]:
# Độ lỗi trên tập test
# Đặt lại baseline_preds
baseline_preds = y_sr.mean()
100 - compute_rr(test_y_sr, full_pipeline.predict(test_X_df), baseline_preds) * 100

37.197251555037035